In [1]:
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
import tqdm
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [6]:
# Read embeddings for entity and relations for animals dataset
ent = pd.read_csv('ConEx_entity_embeddings.csv')
rel = pd.read_csv('ConEx_relation_embeddings.csv')
# print(ent) 
# rel

               Unnamed: 0         0         1         2         3         4  \
0                       0  0.280981 -0.049617 -0.003215 -0.483546 -0.123656   
1                       2  0.259497 -0.124253  0.516882 -0.507972  0.049873   
2    22-rdf-syntax-ns#nil  0.030401 -0.380870  0.387651 -0.021696  0.417615   
3                       4  0.162839  0.023307 -0.004272  0.258616  0.182640   
4                     Air  0.280026  0.329111  0.351390 -0.206114  0.151328   
..                    ...       ...       ...       ...       ...       ...   
136  owl#DatatypeProperty  0.224218 -0.254946  0.285251 -0.331004  0.101589   
137   owl#NamedIndividual -0.008934 -0.033354 -0.210881 -0.478378  0.643270   
138    owl#ObjectProperty  0.331385  0.029724  0.063662 -0.267404  0.271960   
139          owl#Ontology  0.308189  0.348414  0.259391 -0.259898  0.351049   
140       owl#Restriction  0.189910 -0.210566  0.671229  0.012650  0.945052   

            5         6         7         8  ...   

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,22-rdf-syntax-ns#first,0.249388,-0.042922,-0.122261,-0.847049,0.265521,-0.274429,0.199557,0.320650,-0.461976,...,-0.174452,-0.250959,0.013102,-0.720280,-0.839384,0.428789,-0.575384,0.271931,0.039657,0.689879
1,22-rdf-syntax-ns#rest,0.996150,-0.035180,0.148270,0.109840,0.319888,-0.135079,0.572711,-0.007515,-0.169387,...,-0.342145,-0.175748,-0.329169,-0.380170,-0.275613,-0.293082,-0.703566,0.427364,-0.377466,0.998981
2,22-rdf-syntax-ns#type,-0.046696,-0.156884,0.145566,0.240592,0.047457,0.111323,-0.675762,-0.219875,0.263582,...,0.626799,-0.090546,0.241027,0.074701,0.174568,0.020676,0.337071,-0.140033,-0.096334,-0.118992
3,owl#hasValue,-0.290270,-0.367704,-0.098322,-0.260665,-0.219107,0.580961,0.344901,-0.437576,0.536663,...,-0.095163,-0.410139,0.019402,-0.427460,-0.244651,-0.108574,-0.184841,0.215328,0.509272,-0.233955
4,owl#members,-0.974812,-0.194855,-0.282236,-0.130110,-0.262638,0.086012,0.493479,0.023571,0.298599,...,-0.117912,0.397806,-0.207000,0.214214,0.226637,0.072036,-0.015147,-0.174736,-0.434278,-0.359910
5,owl#onProperty,0.113237,0.555730,0.548589,0.174545,-0.260274,0.224632,0.141135,-0.089211,0.193570,...,0.089429,0.441272,-0.011516,0.068045,0.249512,-0.116852,-0.194419,-0.100981,0.220634,-0.286557
6,owl#someValuesFrom,-0.594578,-0.183821,-0.215167,-0.020312,-0.002152,-0.437948,-0.354487,-0.670162,-0.186722,...,0.368161,0.062455,0.629697,0.035124,0.315056,0.411734,0.225164,-0.225102,-0.269512,0.091287
7,owl#unionOf,-0.337723,0.164741,0.558329,-0.395525,-0.115711,-0.600306,0.533303,0.616990,0.239532,...,-0.027793,0.456749,-0.534784,-0.270104,-0.726306,-0.281948,-0.162949,-0.378135,-0.545590,0.164180
8,rdf-schema#domain,0.018777,0.297037,-0.257076,-0.103755,0.113057,-0.832684,0.211413,0.300517,-0.287113,...,0.044454,0.019713,0.008614,-0.485322,-0.236123,-0.273027,0.319627,0.505591,-0.344100,0.024360
9,rdf-schema#range,-0.417240,-0.382403,0.037170,-0.246039,0.205087,-0.405075,0.334612,0.040824,-0.012170,...,0.037547,-0.284791,-0.365411,-0.268734,0.321538,0.267451,0.438465,-0.056473,-0.139085,0.124318


In [4]:
ent.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,0,0.280981,-0.049617,-0.003215,-0.483546,-0.123656,0.564292,0.432948,0.215064,-0.155113,...,0.127948,-0.174766,-0.222745,-0.103216,-0.015835,-0.207434,-0.335994,-0.205721,0.166335,-0.027663
1,2,0.259497,-0.124253,0.516882,-0.507972,0.049873,-0.423409,0.301144,0.041155,-0.204702,...,-0.203218,0.163124,-0.267435,-0.214824,0.091249,-0.208819,-0.376532,-0.185556,0.006606,0.095279
2,22-rdf-syntax-ns#nil,0.030401,-0.380870,0.387651,-0.021696,0.417615,0.254370,0.214374,0.199189,-0.328048,...,-0.338145,-0.275059,0.547209,-0.196628,-0.119073,-0.189195,-0.154224,-0.347735,-0.346463,-0.072465
3,4,0.162839,0.023307,-0.004272,0.258616,0.182640,0.296118,0.342320,0.231499,-0.152109,...,-0.249842,0.077031,-0.241548,-0.195538,-0.170217,-0.172525,-0.368295,-0.326116,-0.198746,0.095087
4,Air,0.280026,0.329111,0.351390,-0.206114,0.151328,0.318364,0.217821,-0.110135,0.014867,...,-0.103858,-0.237071,-0.178284,-0.067466,-0.203515,-0.179597,-0.097360,-0.358156,0.012783,0.263368


In [5]:
ent.shape

(141, 41)

In [6]:
rel.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,22-rdf-syntax-ns#first,0.249388,-0.042922,-0.122261,-0.847049,0.265521,-0.274429,0.199557,0.320650,-0.461976,...,-0.174452,-0.250959,0.013102,-0.720280,-0.839384,0.428789,-0.575384,0.271931,0.039657,0.689879
1,22-rdf-syntax-ns#rest,0.996150,-0.035180,0.148270,0.109840,0.319888,-0.135079,0.572711,-0.007515,-0.169387,...,-0.342145,-0.175748,-0.329169,-0.380170,-0.275613,-0.293082,-0.703566,0.427364,-0.377466,0.998981
2,22-rdf-syntax-ns#type,-0.046696,-0.156884,0.145566,0.240592,0.047457,0.111323,-0.675762,-0.219875,0.263582,...,0.626799,-0.090546,0.241027,0.074701,0.174568,0.020676,0.337071,-0.140033,-0.096334,-0.118992
3,owl#hasValue,-0.290270,-0.367704,-0.098322,-0.260665,-0.219107,0.580961,0.344901,-0.437576,0.536663,...,-0.095163,-0.410139,0.019402,-0.427460,-0.244651,-0.108574,-0.184841,0.215328,0.509272,-0.233955
4,owl#members,-0.974812,-0.194855,-0.282236,-0.130110,-0.262638,0.086012,0.493479,0.023571,0.298599,...,-0.117912,0.397806,-0.207000,0.214214,0.226637,0.072036,-0.015147,-0.174736,-0.434278,-0.359910


In [7]:
rel.shape

(11, 41)

In [8]:
# Create Dataframe of Expressions, individuals and positive and negative examples
columns = ['Expression', 'Individual', 'Example Type']
df = pd.DataFrame(columns=columns)

In [9]:
# Read, a portion of the dataset

# with open('Data.json', 'r') as f:
#     data = json.load(f)
# data
# # filepath = '/home/student117/Documents/aims/project/NCESData/animals/training_data/Data.json'
# # data = json.loads(filepath)

# # df = pd.read_json(filepath)
# # df.head()
# # for expr, val in data.items():
# #     print(expr, val)
#     # for exam in expr:
#     #     print(exam)
# for expr in data:
#     # print(data[expr])
#     i = 0
#     for exam_type in data[expr]:
#         # print(exam_type)
#         i+=1
#         if i == 5:
#             break
#         j = 0
#         for example in data[expr][exam_type]:
#             j+=1
#             if j == 3:
#                 break
#             # print(example)
#             # if exam_type == "positive examples":
#                 # print(data[expr][exam_type])
#             k = 0
#             for ind in data[expr][exam_type]:
#                 k+=1
#                 if k == 5:
#                     break
#                     # 'Expression', 'individual', 'Example Type'
#                 row = {'Expression': expr, 'Example Type': exam_type, 'Individual': ind}
#                 # print(row)
#                 df = df._append(row, ignore_index=True)
#                 # print(row)



In [10]:
# Read whole dataset, takes longer than reading portion of dataset

# for expr in data:
#     # print(data[expr])
#     for exam_type in data[expr]:

#         for example in data[expr][exam_type]:

#             for ind in data[expr][exam_type]:
#                 row = {'Expression': expr, 'Example Type': exam_type, 'Individual': ind}
#                 # print(row)
#                 df = df._append(row, ignore_index=True)
#                 # print(row)


In [11]:
# Save data for future use

# df_all.to_csv('data_all.csv', index=False)
# df_all.shape
# df.shape
df = pd.read_csv('data_all.csv') # uncomment if data already saved
df.head()

,Expression,Individual,Example Type
0,Cat,animals#cat01,positive examples
1,Cat,animals#boy01,negative examples
2,Cat,animals#lizard01,negative examples
3,Cat,animals#snake01,negative examples
4,Cat,animals#trout01,negative examples


In [12]:
print(df.head(20))
print(df['Example Type'].unique())
print(df.shape)
df['Example Type'].unique()

   Expression          Individual       Example Type
0         Cat       animals#cat01  positive examples
1         Cat       animals#boy01  negative examples
2         Cat    animals#lizard01  negative examples
3         Cat     animals#snake01  negative examples
4         Cat     animals#trout01  negative examples
5         Cat       animals#bat01  negative examples
6         Cat     animals#eagle01  negative examples
7         Cat      animals#trex01  negative examples
8         Cat   animals#ostrich01  negative examples
9         Cat       animals#eel01  negative examples
10        Cat       animals#dog01  negative examples
11        Cat    animals#dragon01  negative examples
12        Cat      animals#girl01  negative examples
13        Cat    animals#turtle01  negative examples
14        Cat     animals#shark01  negative examples
15        Cat   animals#dolphin01  negative examples
16        Cat   animals#herring01  negative examples
17        Cat     animals#croco01  negative ex

array(['positive examples', 'negative examples'], dtype=object)

In [13]:
# Encode positive and negative examples

df['Example Encoding'] = df['Example Type'].map({'positive examples': 1, 'negative examples': 0})
df.sample(10)
# df[df['Expression'] == 'Dog ⊔ Platypus']

,Expression,Individual,Example Type,Example Encoding
92097,Eel ⊔ (¬HasGills),animals#eagle01,positive examples,1
131062,Crocodile ⊔ Shark ⊔ Snake,animals#platypus01,negative examples,0
182481,Dog ⊔ Ostrich ⊔ Snake,animals#shark01,negative examples,0
6752,Lizard,animals#platypus01,negative examples,0
105278,Cat ⊔ Dog ⊔ HasEggs,animals#penguin01,positive examples,1
114602,Eagle ⊔ Girl ⊔ Turtle,animals#penguin01,negative examples,0
72780,Girl ⊔ Snake,animals#ostrich01,negative examples,0
203837,Trout ⊔ (Homeothermic ⊓ (Platypus ⊔ (¬Bat))),animals#eel01,negative examples,0
163173,Crocodile ⊔ Girl ⊔ Lizard,animals#ostrich01,negative examples,0
13001,¬Trout,animals#penguin01,positive examples,1


In [14]:
# Extract atomic classes names from dataset

constructors = ['¬', '⊔', '⊓']
# i = 0
# atomic_classes = []
# for expr in data:
#   atomic_classes.append(expr)
#   # print(expr)
#   i+=1
#   if i == 24:
#     break
atomic_classes = ['Homeothermic', 'HasEggs', 'HasMilk', 'HasGills', 'Eel', 'Ostrich', 'Trout', 'Crocodile', 'Girl', 'Lizard', 'Shark', 'Bat', 'Cat', 'Dolphin', 'Eagle', 'Dog', 'Snake', 'Herring', 'Boy', 'Turtle', 'Penguin', 'T-Rex', 'Platypus', 'Dragon']
len(atomic_classes)

24

In [15]:
# Extract atomic classes for training

atomic = df[df['Expression'].isin(atomic_classes)]
atomic

,Expression,Individual,Example Type,Example Encoding
0,Cat,animals#cat01,positive examples,1
1,Cat,animals#boy01,negative examples,0
2,Cat,animals#lizard01,negative examples,0
3,Cat,animals#snake01,negative examples,0
4,Cat,animals#trout01,negative examples,0
...,...,...,...,...
8183,Dog,animals#girl01,negative examples,0
8184,Dog,animals#cat01,negative examples,0
8185,Dog,animals#shark01,negative examples,0
8186,Dog,animals#croco01,negative examples,0


In [16]:
# merge classes and individuals with their embdenings

atomic_ent = pd.merge(atomic, ent, left_on='Individual', right_on='Unnamed: 0', how='inner')
# atomic_ent.columns
atomic_ent = pd.merge(atomic_ent, ent, left_on="Expression", right_on='Unnamed: 0', how='inner', suffixes=['_ind', '_cla'])
# a.head()

In [17]:
# atomic_data = atomic_ent[['Expression', 'Individual', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 'Example Encoding']]
atomic_data = atomic_ent[['Expression', 'Individual', '0_ind', '1_ind', '2_ind', '3_ind', '4_ind', '5_ind', '6_ind', '7_ind', '8_ind', '9_ind', '10_ind', '11_ind', '12_ind', '13_ind', '14_ind', '15_ind', '16_ind', '17_ind', '18_ind', '19_ind', '20_ind', '21_ind', '22_ind', '23_ind', '24_ind', '25_ind', \
                          '26_ind', '27_ind', '28_ind', '29_ind', '30_ind', '31_ind', '32_ind', '33_ind', '34_ind', '35_ind', '36_ind', '37_ind', '38_ind', '39_ind', '0_cla', '1_cla', '2_cla', '3_cla', '4_cla', '5_cla', '6_cla', '7_cla', '8_cla', '9_cla', '10_cla', '11_cla', '12_cla', '13_cla', '14_cla', '15_cla', '16_cla', '17_cla', '18_cla', '19_cla', '20_cla', '21_cla', '22_cla', '23_cla', '24_cla', '25_cla', '26_cla', '27_cla', '28_cla', '29_cla', '30_cla', '31_cla', '32_cla', '33_cla', '34_cla', '35_cla', '36_cla', '37_cla', '38_cla', '39_cla', 'Example Encoding']]

atomic_data

,Expression,Individual,0_ind,1_ind,2_ind,3_ind,4_ind,5_ind,6_ind,7_ind,...,31_cla,32_cla,33_cla,34_cla,35_cla,36_cla,37_cla,38_cla,39_cla,Example Encoding
0,Cat,animals#cat01,0.183570,0.064570,0.119314,-0.220391,0.185930,0.059797,0.320332,0.277950,...,-0.317788,-0.006029,-0.285176,0.176120,-0.070169,-0.269090,-0.445585,-0.012556,0.332329,1
1,Cat,animals#boy01,0.326895,-0.017734,0.072276,-0.148582,0.154523,0.143247,0.258985,0.121672,...,-0.317788,-0.006029,-0.285176,0.176120,-0.070169,-0.269090,-0.445585,-0.012556,0.332329,0
2,Cat,animals#lizard01,0.348107,-0.256041,0.186654,-0.198851,0.320173,0.172015,0.128844,0.057844,...,-0.317788,-0.006029,-0.285176,0.176120,-0.070169,-0.269090,-0.445585,-0.012556,0.332329,0
3,Cat,animals#snake01,0.341165,0.044748,0.248957,-0.215969,0.217901,0.206946,0.298251,0.171259,...,-0.317788,-0.006029,-0.285176,0.176120,-0.070169,-0.269090,-0.445585,-0.012556,0.332329,0
4,Cat,animals#trout01,0.115120,-0.028380,0.157293,-0.273941,0.275957,0.136321,0.229726,0.066404,...,-0.317788,-0.006029,-0.285176,0.176120,-0.070169,-0.269090,-0.445585,-0.012556,0.332329,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8183,Dog,animals#girl01,0.231076,-0.074579,0.161141,-0.239554,0.166832,0.142751,0.275325,0.102900,...,0.169880,0.206478,-0.285225,-0.196411,-0.136789,-0.248808,-0.360837,-0.111585,0.350478,0
8184,Dog,animals#cat01,0.183570,0.064570,0.119314,-0.220391,0.185930,0.059797,0.320332,0.277950,...,0.169880,0.206478,-0.285225,-0.196411,-0.136789,-0.248808,-0.360837,-0.111585,0.350478,0
8185,Dog,animals#shark01,0.265107,0.035628,0.237405,-0.208801,0.326406,0.151469,0.338959,0.051302,...,0.169880,0.206478,-0.285225,-0.196411,-0.136789,-0.248808,-0.360837,-0.111585,0.350478,0
8186,Dog,animals#croco01,0.194329,-0.162931,0.175191,-0.272689,0.295109,0.161218,0.429787,0.174258,...,0.169880,0.206478,-0.285225,-0.196411,-0.136789,-0.248808,-0.360837,-0.111585,0.350478,0


In [1]:
# # X = adc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 'Homeothermic', 'HasEggs', 'HasMilk', 'HasGills', 'Eel', 'Ostrich', 'Trout', 'Crocodile', 'Girl', 'Lizard', 'Shark', 'Bat', 'Cat', 'Dolphin', 'Eagle', 'Dog', 'Snake', 'Herring', 'Boy', 'Turtle', 'Penguin', 'T-Rex', 'Platypus', 'Dragon']]
# # X = adc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']]
# X = atomic_data[['0_ind', '1_ind', '2_ind', '3_ind', '4_ind', '5_ind', '6_ind', '7_ind', '8_ind', '9_ind', '10_ind', '11_ind', '12_ind', '13_ind', '14_ind', '15_ind', '16_ind', '17_ind', '18_ind', '19_ind', '20_ind', '21_ind', '22_ind', '23_ind', '24_ind', '25_ind', '26_ind', '27_ind', '28_ind', '29_ind', '30_ind', '31_ind', '32_ind', '33_ind', '34_ind', '35_ind', '36_ind', '37_ind', '38_ind', '39_ind', '0_cla', '1_cla', '2_cla', '3_cla', '4_cla', '5_cla', '6_cla', '7_cla', '8_cla', '9_cla', '10_cla', '11_cla', '12_cla', '13_cla', '14_cla', '15_cla', '16_cla', '17_cla', '18_cla', '19_cla', '20_cla', '21_cla', '22_cla', '23_cla', '24_cla', '25_cla', '26_cla', '27_cla', '28_cla', '29_cla', '30_cla', '31_cla', '32_cla', '33_cla', '34_cla', '35_cla', '36_cla', '37_cla', '38_cla', '39_cla']]
# y = atomic_data['Example Encoding']
# X#.shape
# # y

In [ ]:
import pandas as pd
fam = pd.read_csv('../python/NCESData/family/embeddings/ConEx_entity_embeddings.csv')

In [ ]:
fam

In [ ]:
fam['Unnamed: 0'] = fam['Unnamed: 0'].apply(lambda x: x.replace('family#', ''))
fam

In [ ]:
fam.set_index(fam.columns[0], inplace=True)
fam